In [2]:
import cleverhans
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.utils import np_utils
# new Data preparation

# Training Dataset
data = pd.read_csv("./dataset/UNSW_2018_IoT_Botnet_Final_10_best_Training.csv")

x_features = data[['seq','stddev','N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'N_IN_Conn_P_DstIP',
       'drate', 'srate', 'max', 'category', 'subcategory']]

# Label Encoding the target columns
le = LabelEncoder()
x_features['category'] = le.fit_transform(x_features['category'])
x_features['subcategory'] = le.fit_transform(x_features['subcategory'])

x = x_features.values
x = preprocessing.scale(x)
y = data['attack'].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
# Define model

model = tf.keras.Sequential([
        tf.keras.layers.Dense(200, input_dim=x.shape[1], activation=tf.nn.relu),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dense(200, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(np.unique(y))),
        # We seperate the activation layer to be able to access
        # the logits of the previous layer later
        tf.keras.layers.Activation(tf.nn.softmax)
        ])

model.compile(optimizer='adam',
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])

In [3]:
# Train Classification model

y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

model.fit(X_train, y_train_cat, epochs=10, validation_split=0.2)

test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print('Test accuracy:', test_acc)

Epoch 1/10
58697/58697 [==============================] - 149s 3ms/step - loss: 0.0023 - accuracy: 0.9999 - val_loss: 0.0033 - val_accuracy: 1.0000
Epoch 2/10
58697/58697 [==============================] - 163s 3ms/step - loss: 3.9948e-04 - accuracy: 1.0000 - val_loss: 0.0028 - val_accuracy: 1.0000
Epoch 3/10
58697/58697 [==============================] - 150s 3ms/step - loss: 5.9921e-04 - accuracy: 1.0000 - val_loss: 0.0024 - val_accuracy: 1.0000
Epoch 4/10
58697/58697 [==============================] - 158s 3ms/step - loss: 6.8712e-04 - accuracy: 1.0000 - val_loss: 2.7956e-04 - val_accuracy: 1.0000
Epoch 5/10
58697/58697 [==============================] - 156s 3ms/step - loss: 1.5962e-04 - accuracy: 1.0000 - val_loss: 9.9860e-04 - val_accuracy: 1.0000
Epoch 6/10
58697/58697 [==============================] - 156s 3ms/step - loss: 3.0442e-04 - accuracy: 1.0000 - val_loss: 1.6701e-04 - val_accuracy: 1.0000
Epoch 7/10
58697/58697 [==============================] - 161s 3ms/step - loss: 

In [4]:
# Original Confusion Matrix of Model

from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
cm_org = confusion_matrix(y_test, y_pred)
print("*"*50)
print("Original confusion matrix")
print(cm_org)

18343/18343 [==============================] - 17s 932us/step
**************************************************
Original confusion matrix
[[     4     80]
 [359238 227642]]


In [5]:
# Import the attack
from cleverhans.future.tf2.attacks import fast_gradient_method

#The attack requires the model to ouput the logits
logits_model = tf.keras.Model(model.input, model.layers[-1].output)

epsilon = 0.1
adv_fgsm_x = fast_gradient_method(logits_model, X_test, epsilon, np.inf, targeted=False)

# Attack sample confusion matrix
y_pred = model.predict(adv_fgsm_x)
y_pred=np.argmax(y_pred, axis=1)
cm_adv = confusion_matrix(y_test, y_pred)
print("*"*20)
print("Attacked confusion matrix")
print(cm_adv)

18343/18343 [==============================] - 22s 1ms/step
********************
Attacked confusion matrix
[[     7     77]
 [311728 275152]]


In [9]:
# Adversarial sample generated
print(adv_fgsm_x)

tf.Tensor(
[[-0.53913366  1.20519677  0.71545303 ...  0.74155357  0.75246724
   0.84493152]
 [ 0.29728963  1.32828562 -0.76055039 ...  0.61546879 -0.83068184
   0.84493152]
 [ 0.01738669 -1.03265755  0.71545303 ...  0.32612128 -0.83068184
  -0.86685956]
 ...
 [-1.2337648   0.50046989  0.71545303 ...  0.60909558 -0.83068184
   0.84493152]
 [-1.55738237  0.07869345  0.71545303 ...  0.55761283 -0.83068184
   0.84493152]
 [ 0.24483999  0.83125896 -0.7195503  ...  0.31449904  0.75246724
   0.84493152]], shape=(586964, 12), dtype=float64)


In [6]:
# Adversarial training confusion matrix
adv_fgsm_x_train = fast_gradient_method(logits_model, X_train, epsilon, np.inf, targeted=False)
adv_fgsm_x_test = fast_gradient_method(logits_model, X_test, epsilon, np.inf, targeted=False)

concat_adv_x = np.concatenate([X_train, adv_fgsm_x_train])
concat_y_train = np.concatenate([y_train_cat, y_train_cat])

# Adversarial training
model.fit(concat_adv_x, concat_y_train, epochs=10, validation_data=(adv_fgsm_x_test, y_test_cat))
y_pred = model.predict(adv_fgsm_x_test)
y_pred = np.argmax(y_pred, axis=1)
# Print out Confusion Matrix
cm_adv = confusion_matrix(y_test, y_pred)
print("*"*20)
print("Adversarial training confusion matrix")
print(cm_adv)

Epoch 1/10
146741/146741 [==============================] - 366s 2ms/step - loss: 0.0057 - accuracy: 1.0000 - val_loss: 0.0036 - val_accuracy: 1.0000
Epoch 2/10
146741/146741 [==============================] - 354s 2ms/step - loss: 0.0041 - accuracy: 1.0000 - val_loss: 1.5897e-04 - val_accuracy: 1.0000
Epoch 3/10
146741/146741 [==============================] - 350s 2ms/step - loss: 0.0018 - accuracy: 1.0000 - val_loss: 0.0043 - val_accuracy: 1.0000
Epoch 4/10
146741/146741 [==============================] - 340s 2ms/step - loss: 9.7766e-04 - accuracy: 1.0000 - val_loss: 1.2938e-04 - val_accuracy: 1.0000
Epoch 5/10
146741/146741 [==============================] - 388s 3ms/step - loss: 2.7376e-04 - accuracy: 1.0000 - val_loss: 1.5582e-04 - val_accuracy: 1.0000
Epoch 6/10
146741/146741 [==============================] - 394s 3ms/step - loss: 6.1251e-04 - accuracy: 1.0000 - val_loss: 9.6285e-05 - val_accuracy: 1.0000
Epoch 7/10
146741/146741 [==============================] - 408s 3ms/ste